# Timeseries forecasting using VARMA

This notebook outlines how we have used machine learning for forecasting elephant movement. We have used the VARMA method, implemented using the VARMAX model found in the statsmodels package.

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.varmax import VARMAX, VARMAXResults

In [3]:
from read_from_db import read_from_db

df_main, df_time = read_from_db()

## Data wrangling

The time series data we are working with is not immeadietly usable in the VARMA method because the method requires a constant time frequency throughout the data. In our dataset the datapoints have been collected at various times throughout different days. Therefore we summarize, for each elephant and each day, the mean of latitude, longitude and temperature into the df_daily_avg DataFrame.

Some of the elephants are missing values for certain days. We therefore pick out three of the elephants (that have many days without missing values in common) and a time period where they have no missing values. The choice was made by simple inspection of the data in a text editor.

We want to train a model on data from these three elephants. To do that we need to reshape the data so that we have one column for each elephant and each of the three dependant variables, latitude, longitude and temperature.

In [82]:
df_merged = pd.merge(df_main, df_time, how='left', on='time_stamp')
df_merged = df_merged[['latitude', 'longitude', 'temperature', 'tag_id', 'year', 'month', 'day']]

tag_ids = ['AM91', 'AM93', 'AM99']
df_daily_avg = df_merged.query(f'tag_id in {tag_ids}').groupby(by=['tag_id', 'year', 'month', 'day']).mean()

df_pivot = (df_daily_avg
                .unstack(level=0) # pivot tag_id:s to columns
                .swaplevel(axis=1).reindex(tag_ids, axis=1, level=0) # fix multilevel column names
)
df_filtered = df_pivot.reset_index().iloc[137:444].reset_index() # filter the chosen days
df_to_model = df_filtered[['AM91', 'AM93', 'AM99']] # remove date columns for insertion into model

In [129]:
df_filtered.to_csv('./data/ML/pivoted_data.csv')

We are now finally ready to train the model. We split of the last days in the dataset, so that we can compare forecasts with the true continuation of the time series.

In [130]:
split_index = int(len(df_to_model)*0.9)
train_data = df_to_model[:split_index]
test_data = df_to_model[split_index:]

# fit model
model = VARMAX(train_data, order=(2, 1))
model_fit = model.fit(disp=False)

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [150]:
model_fit.save('./data/ML/varma_2_1_model')

The following code is used to make som plots, and is the basis for the plotting used in the main app.

In [151]:
model_fit = VARMAXResults.load('./data/ML/varma_2_1_model')

In [152]:
tag_used = 'AM99'
nr_days = 3
lag = 7

yhat = model_fit.forecast(steps=nr_days)
forecast = yhat[[f'{tag_used}_latitude', f'{tag_used}_longitude']]
print(forecast)
truth = df_filtered.iloc[len(train_data)-lag:len(train_data)+nr_days][tag_used]
print(truth)
print(truth.iloc[lag:])

     AM99_latitude  AM99_longitude
276     -24.236872       31.654765
277     -24.234021       31.649851
278     -24.236665       31.652620
      latitude  longitude  temperature
269 -24.259609  31.654337    26.000000
270 -24.241704  31.663251    28.704545
271 -24.242508  31.661738    29.702128
272 -24.258388  31.653639    27.738095
273 -24.258013  31.658922    30.255319
274 -24.252172  31.674940    24.680851
275 -24.247203  31.666862    24.608696
276 -24.250960  31.652615    28.936170
277 -24.238475  31.661332    30.282609
278 -24.225781  31.675317    29.000000
      latitude  longitude  temperature
276 -24.250960  31.652615    28.936170
277 -24.238475  31.661332    30.282609
278 -24.225781  31.675317    29.000000


In [153]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = truth['longitude'],
    lat = truth['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = forecast[f'{tag_used}_longitude'],
    lat = forecast[f'{tag_used}_latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = truth.iloc[lag:]['longitude'],
    lat = truth.iloc[lag:]['latitude'],
    marker = {'size': 10}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': forecast[f'{tag_used}_longitude'].mean(), 'lat': forecast[f'{tag_used}_latitude'].mean()},
        'style': "stamen-terrain",
        'zoom': 12})

fig.show()